In [37]:
import numpy as np
import pandas as pd
import sklearn as sk
import sys
#hyper-parameters
INPUT_LAYER_SIZE=784
OUTPUT_LAYER_SIZE=10
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [2]:
train=pd.read_csv('./data/train.csv')
val=pd.read_csv('./data/val.csv')
test=pd.read_csv('./data/test.csv')

In [34]:
training_data_class =train.iloc[:,-1].copy().as_matrix()
training_data=train.iloc[:,1:785].copy().as_matrix()/255.0

In [35]:
validating_data_class =val.iloc[:,-1].copy().as_matrix()
validating_data=val.iloc[:,1:785].copy().as_matrix()/255.0

In [36]:
training_data=sk.preprocessing.scale(training_data)
validating_data=sk.preprocessing.scale(validating_data)

In [43]:
print (training_data[:,5])

[-0.07072376  0.10359759 -0.07072376 ..., -0.07072376 -0.07072376
  0.10359759]


In [23]:
clf = LinearDiscriminantAnalysis()
clf.fit(training_data,training_data_class)


LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)

In [24]:
training_data=clf.transform(training_data)
validating_data=clf.transform(validating_data)

In [25]:
training_data=sk.preprocessing.normalize(training_data)
validating_data=sk.preprocessing.normalize(validating_data)

In [26]:
#training_data=np.vstack((training_data,validating_data))
#training_data_class=np.concatenate((training_data_class,validating_data_class))
training_data.shape


(55000, 9)

In [38]:
def create_output(class_labels):
    '''returns one-hot vectors'''
    output=np.zeros((len(class_labels),10))
    for i in range(len(class_labels)):
        output[i,class_labels[i]]=1
    return output
def sigmoid(value):
    return 1/(1+np.exp(-value))
def tanh(value):
    return np.tanh(value)
def relu(value):
    return [max(0,i) for i in value]
def relu_dife(value):
    if value>0:
        return 1
    return 0
def relu_dif(value):
    return [relu_dife(i) for i in value]
def sigmoid_dif(value):
    return sigmoid(value)*(1-sigmoid(value))
def tanh_dif(value):
    return 1-tanh(value)*tanh(value)
def softmax(vector):
    vector=np.array(vector,dtype=np.float64)
    if sum(vector)!=sum(vector):
        #print vector
        sys.exit()
    num= np.exp(vector)
    
    return num/np.sum(num)
def paraCopy(w,b,multiplier=1):
    return [i*multiplier for i in w],[i*multiplier for i in b]
def paraAdd(A,B):
    a,b=A
    c,d=B
    return [i+j for i,j in zip(a,c)],[i+j for i,j in zip(b,d)]
def cliper(A):
    return A/np.linalg.norm(A)
#def cliper(A,B):
#    return [i/np.linalg.norm(i) for i in A],[j/np.linalg.norm(j) for j in B]

In [39]:
def shaper(lst):
    for i in lst:
        print 'shape -',i.shape
def loss(list1,list2,loss_type):
    if loss_type=='ce': #ce 
        return -np.sum([j*np.log2(i) for i,j in zip(list1,list2)])
    if loss_type=='sq': #sq
        return 0.5*np.sum([(i-j)**2 for i,j in zip(list1,list2)])
def dumpModel(model):
    kp=0
    for i,j in zip(model.weights,model.biases):
        np.save('./temp/weights_'+str(kp),i)
        np.save('./temp/biases_'+str(kp),j)
        kp+=1


In [40]:
class NN_Model:
    def __init__(self,lr=0.01,momentum=0,hidden_layers=(100,),activation='sigmoid',loss='sq',
                 opt='gd',batch_size=1,anneal=False,save_dir='./temp/',expt_dir='./temp/',max_iter=50,ae_mode=False,
                noise=0.0,Lambda=0):
        self.lr=lr
        self.momentum=momentum
        self.layers=(INPUT_LAYER_SIZE,)+hidden_layers+(OUTPUT_LAYER_SIZE,)
        if activation=='sigmoid':
            self.activation=sigmoid
            self.activation_diff=sigmoid_dif
        elif activation=='tanh':
            self.activation=tanh
            self.activation_diff=tanh_dif
        elif activation=='relu':
            self.activation=relu
            self.activation_diff=relu_dif
        else:
            print 'Error : activation function not found'
        self.activation_name=activation        
        self.p_noise=noise
        self.loss=loss
        self.opt=opt
        self.batch_size=batch_size
        self.anneal=anneal
        self.save_dir=save_dir
        self.expt_dir=expt_dir
        self.num_layers=len(self.layers)
        self.max_iter=max_iter
        self.ae_mode=ae_mode
        self.lam=Lambda
    
    def __forward_propagation(self):
        h_set=[]
        a_set=[]
        h=self.input_data[TRAINER]
        h=h+np.random.normal(size=h.shape)*np.random.choice([0, 1], size=h.shape, p=[1-self.p_noise,self.p_noise])

        h_set.append(h) #experimental
        L=self.num_layers-2 # 2 are input and output layers
        for k in range(L): 
            a=self.biases[k]+np.matmul(self.weights[k],h)
            
            h=self.activation(a)
            if self.activation_name=='relu':
                h=cliper(h)
            a_set.append(a)
            h_set.append(h)
        a=self.biases[L]+np.matmul(self.weights[L],h)
        
        a_set.append(a)
        #print a
        y=softmax(a)
        #sys.exit()
        #h_set.append(y) #experimental
        #print h.shape
            
        return h_set,a_set,y
    
    def __forward_propagation_test(self,h):
        L=self.num_layers-2 # 2 are input and output layers
        for k in range(L): 
            a=self.biases[k]+np.matmul(self.weights[k],h)
            h=self.activation(a)
            if self.activation_name=='relu':
                h=cliper(h)
            
        a=self.biases[L]+np.matmul(self.weights[L],h)
        if self.ae_mode==False:
            y=softmax(a)
        else:
            y=a
        #h_set.append(y) #experimental
        #print h.shape
            
        return y
    
    
    def __back_propagation(self,h,a,y):
        L=self.num_layers-2
        dWeights=[]
        dBiases=[]
        if self.loss=='ce':
            daL_loss=-(self.output_data[TRAINER]-y) # for cross-entropy loss function
        elif self.loss=='sq':
            daL_loss=np.array([2*sum([(y[i]-self.output_data[TRAINER][i])*y[i]*((i==j)*1-y[j]) for i in range(len(y))]) for j in range(len(y))])
        else :
            print 'Wrong loss function'
            return
        dA_loss=daL_loss
        for k in range(L,-1,-1):
            #print 'k=',k
            dW_loss=np.outer(dA_loss,h[k]).T
            dB_loss=dA_loss
            
            if k!=0:
                dH_loss=np.matmul(self.weights[k].T,dA_loss)
            
                dA_loss=dH_loss*self.activation_diff(a[k-1])
            
            
            dWeights=[dW_loss.T]+dWeights
            dBiases=[dB_loss.T]+dBiases
            #print dA_loss.shape
            #return _,_
            
            
        return dWeights,dBiases
    
    def __update(self,dWeights,dBiases):
        for i in range(len(self.weights)):
            self.weights[i]=self.weights[i]-(dWeights[i]+self.lam*np.absolute(self.weights[i]))
            self.biases[i]=self.biases[i]-(dBiases[i]+self.lam*np.absolute(self.biases[i]))

    
    
    
    def __train(self):
        global TRAINER
        for i in range(self.max_iter):
            if i%5==0 and i!=0 and self.anneal==True:
                self.lr/=2
            
            accuracy=0
            if self.batch_size==1:
                self.batch_size=len(training_data)
            
            sets=len(training_data)/self.batch_size # number of batches
                
            oldDWeights,oldDBiases=paraCopy(self.weights,self.biases,0) # for momentum
            for j in range(sets):
                #print 'SET=',j
                los=0
                dWeights,dBiases=paraCopy(self.weights,self.biases,0)
                for TRAINER in range(j*self.batch_size,(j+1)*self.batch_size):
                    #print 'TRAINER=',TRAINER
                    h,a,y=self.__forward_propagation()

                    
                    dWeights,dBiases=paraAdd(self.__back_propagation(h,a,y),
                                         paraCopy(dWeights,dBiases,1))
                    
                    los+=loss(y,self.output_data[TRAINER],self.loss)

                dWeights,dBiases=paraAdd(paraCopy(oldDWeights,oldDBiases,self.momentum),
                                         paraCopy(dWeights,dBiases,self.lr))

                self.__update(dWeights,dBiases)


                
                oldDWeights,oldDBiases=paraCopy(dWeights,dBiases,1)

                #print np.argmax(y),self.raw_class_labels[TRAINER]
                #if np.argmax(y)==self.raw_class_labels[TRAINER]:

                 #   accuracy+=1
                print 'Epoch : ',i,'Step : ',j,'loss : ',los
                #print 'acc=',accuracy*1.0/sets
            #print 'validation loss ',sum([loss(i,)])

    def fit(self,X,Y):
        self.input_data=X
        if self.ae_mode== False:
            self.output_data=create_output(Y)
        else:
            self.output_data=Y
        self.weights=[]
        self.biases=[]
        self.raw_class_labels=Y
        for i in range(self.num_layers-1):
            self.weights.append(np.random.uniform(low=-1.0,high=1.0,size=(self.layers[i+1],self.layers[i])))
            self.biases.append(np.random.uniform(low=-1.0,high=1.0,size=(self.layers[i+1])))
            
            #print self.biases[-1].shape
        self.__train()
        
    def resume(self,X,Y):
        self.input_data=X
        self.output_data=create_output(Y)
        self.weights=[]
        self.biases=[]
        self.raw_class_labels=Y
        for i in range(self.num_layers-1):
            self.weights.append(np.load('./temp/weights_'+str(i)+'.npy'))
            self.biases.append(np.load('./temp/biases_'+str(i)+'.npy'))
            #print self.weights[-1].shape
        self.__train()
        
        
    def predict(self,X):
        return [np.argmax(self.__forward_propagation_test(i)) for i in X]    
        
    

In [41]:
model=NN_Model(hidden_layers=(128,128),lr=0.001,max_iter=20,momentum=0.5,batch_size=250,anneal=True,loss='ce',
               activation='tanh',ae_mode=False,noise=0.0,Lambda=0.0)
training_data=np.array(training_data)

In [ ]:
model.fit(training_data,training_data_class)
#model.resume(training_data,training_data_class)
#model.build()

Epoch :  0 Step :  0 loss :  3275.61215564
Epoch :  0 Step :  1 loss :  2568.59560899
Epoch :  0 Step :  2 loss :  1613.36949865
Epoch :  0 Step :  3 loss :  1552.23079299
Epoch :  0 Step :  4 loss :  1370.87332768
Epoch :  0 Step :  5 loss :  1297.54507895
Epoch :  0 Step :  6 loss :  872.768822322
Epoch :  0 Step :  7 loss :  981.552584105
Epoch :  0 Step :  8 loss :  1021.33712178
Epoch :  0 Step :  9 loss :  908.843558125
Epoch :  0 Step :  10 loss :  789.47776086
Epoch :  0 Step :  11 loss :  1004.57908755
Epoch :  0 Step :  12 loss :  880.080083942
Epoch :  0 Step :  13 loss :  819.169437261
Epoch :  0 Step :  14 loss :  779.692550997
Epoch :  0 Step :  15 loss :  921.418867403
Epoch :  0 Step :  16 loss :  838.945175211
Epoch :  0 Step :  17 loss :  735.891182184
Epoch :  0 Step :  18 loss :  632.570345187
Epoch :  0 Step :  19 loss :  603.145650158
Epoch :  0 Step :  20 loss :  798.993331815
Epoch :  0 Step :  21 loss :  771.119003866
Epoch :  0 Step :  22 loss :  795.617337665

Epoch :  0 Step :  185 loss :  292.141802638
Epoch :  0 Step :  186 loss :  297.155357984
Epoch :  0 Step :  187 loss :  321.059549136
Epoch :  0 Step :  188 loss :  421.972238994
Epoch :  0 Step :  189 loss :  298.789152438
Epoch :  0 Step :  190 loss :  328.294398839
Epoch :  0 Step :  191 loss :  255.352253224
Epoch :  0 Step :  192 loss :  235.905426999
Epoch :  0 Step :  193 loss :  283.245142796
Epoch :  0 Step :  194 loss :  289.911481078
Epoch :  0 Step :  195 loss :  299.787717165
Epoch :  0 Step :  196 loss :  256.914059973
Epoch :  0 Step :  197 loss :  320.831857847
Epoch :  0 Step :  198 loss :  308.016094291
Epoch :  0 Step :  199 loss :  363.460084222
Epoch :  0 Step :  200 loss :  335.357719164
Epoch :  0 Step :  201 loss :  317.264873789
Epoch :  0 Step :  202 loss :  309.39121332
Epoch :  0 Step :  203 loss :  276.730516818
Epoch :  0 Step :  204 loss :  352.518319051
Epoch :  0 Step :  205 loss :  345.323215263
Epoch :  0 Step :  206 loss :  319.495527924
Epoch :  0 

Epoch :  1 Step :  150 loss :  207.946251662
Epoch :  1 Step :  151 loss :  230.229712423
Epoch :  1 Step :  152 loss :  254.019994471
Epoch :  1 Step :  153 loss :  191.214405858
Epoch :  1 Step :  154 loss :  244.837957942
Epoch :  1 Step :  155 loss :  245.150991303
Epoch :  1 Step :  156 loss :  250.850356249
Epoch :  1 Step :  157 loss :  214.075891244
Epoch :  1 Step :  158 loss :  235.013621098
Epoch :  1 Step :  159 loss :  213.670643019
Epoch :  1 Step :  160 loss :  213.880585071
Epoch :  1 Step :  161 loss :  222.815179721
Epoch :  1 Step :  162 loss :  222.721295883
Epoch :  1 Step :  163 loss :  233.276255125
Epoch :  1 Step :  164 loss :  190.466935685
Epoch :  1 Step :  165 loss :  232.657651562
Epoch :  1 Step :  166 loss :  195.177658261
Epoch :  1 Step :  167 loss :  197.114613018
Epoch :  1 Step :  168 loss :  220.967494396
Epoch :  1 Step :  169 loss :  241.414459187
Epoch :  1 Step :  170 loss :  214.041061295
Epoch :  1 Step :  171 loss :  228.614700795
Epoch :  1

Epoch :  2 Step :  115 loss :  214.307239628
Epoch :  2 Step :  116 loss :  207.433345115
Epoch :  2 Step :  117 loss :  215.911107313
Epoch :  2 Step :  118 loss :  220.714660756
Epoch :  2 Step :  119 loss :  190.15202346
Epoch :  2 Step :  120 loss :  195.838599562
Epoch :  2 Step :  121 loss :  165.010550908
Epoch :  2 Step :  122 loss :  212.040449235
Epoch :  2 Step :  123 loss :  195.405374833
Epoch :  2 Step :  124 loss :  182.372703414
Epoch :  2 Step :  125 loss :  192.387401759
Epoch :  2 Step :  126 loss :  186.098199916
Epoch :  2 Step :  127 loss :  165.488807272
Epoch :  2 Step :  128 loss :  188.737269478
Epoch :  2 Step :  129 loss :  197.821402929
Epoch :  2 Step :  130 loss :  158.838442528
Epoch :  2 Step :  131 loss :  227.321391912
Epoch :  2 Step :  132 loss :  143.031919541
Epoch :  2 Step :  133 loss :  213.663461975
Epoch :  2 Step :  134 loss :  217.996287872
Epoch :  2 Step :  135 loss :  226.379555027
Epoch :  2 Step :  136 loss :  216.389694992
Epoch :  2 

Epoch :  3 Step :  80 loss :  154.201947759
Epoch :  3 Step :  81 loss :  184.961501522
Epoch :  3 Step :  82 loss :  194.282425786
Epoch :  3 Step :  83 loss :  172.585005669
Epoch :  3 Step :  84 loss :  156.879658575
Epoch :  3 Step :  85 loss :  202.755691506
Epoch :  3 Step :  86 loss :  137.332086965
Epoch :  3 Step :  87 loss :  159.996531603
Epoch :  3 Step :  88 loss :  196.479659413
Epoch :  3 Step :  89 loss :  133.726065778
Epoch :  3 Step :  90 loss :  169.773717279
Epoch :  3 Step :  91 loss :  207.419862435
Epoch :  3 Step :  92 loss :  173.13224964
Epoch :  3 Step :  93 loss :  190.233935243
Epoch :  3 Step :  94 loss :  152.51831558
Epoch :  3 Step :  95 loss :  161.709940993
Epoch :  3 Step :  96 loss :  161.640362293
Epoch :  3 Step :  97 loss :  168.155189427
Epoch :  3 Step :  98 loss :  178.435424688
Epoch :  3 Step :  99 loss :  201.208427353
Epoch :  3 Step :  100 loss :  133.792006071
Epoch :  3 Step :  101 loss :  146.458448111
Epoch :  3 Step :  102 loss :  1

...

Epoch :  9 Step :  56 loss :  143.947140662
Epoch :  9 Step :  57 loss :  128.500245569
Epoch :  9 Step :  58 loss :  105.356348891
Epoch :  9 Step :  59 loss :  124.235100757
Epoch :  9 Step :  60 loss :  111.996683474
Epoch :  9 Step :  61 loss :  122.547893933
Epoch :  9 Step :  62 loss :  129.265757865
Epoch :  9 Step :  63 loss :  103.604953925
Epoch :  9 Step :  64 loss :  130.751345321
Epoch :  9 Step :  65 loss :  101.87806624
Epoch :  9 Step :  66 loss :  138.421129713
Epoch :  9 Step :  67 loss :  125.203139898
Epoch :  9 Step :  68 loss :  101.42558577
Epoch :  9 Step :  69 loss :  153.527933582
Epoch :  9 Step :  70 loss :  149.343421264
Epoch :  9 Step :  71 loss :  124.143148206
Epoch :  9 Step :  72 loss :  141.328855283
Epoch :  9 Step :  73 loss :  127.143242115
Epoch :  9 Step :  74 loss :  100.708482938
Epoch :  9 Step :  75 loss :  130.356964464
Epoch :  9 Step :  76 loss :  178.194286481
Epoch :  9 Step :  77 loss :  120.426643458
Epoch :  9 Step :  78 loss :  87.1

Epoch :  10 Step :  20 loss :  146.949248589
Epoch :  10 Step :  21 loss :  150.616378719
Epoch :  10 Step :  22 loss :  118.367503906
Epoch :  10 Step :  23 loss :  94.5477025728
Epoch :  10 Step :  24 loss :  111.873364899
Epoch :  10 Step :  25 loss :  123.014733356
Epoch :  10 Step :  26 loss :  92.6976456737
Epoch :  10 Step :  27 loss :  98.6919459819
Epoch :  10 Step :  28 loss :  109.173946949
Epoch :  10 Step :  29 loss :  99.8795447075
Epoch :  10 Step :  30 loss :  125.490011382
Epoch :  10 Step :  31 loss :  112.082234576
Epoch :  10 Step :  32 loss :  107.97069446
Epoch :  10 Step :  33 loss :  110.412168276
Epoch :  10 Step :  34 loss :  107.133328173
Epoch :  10 Step :  35 loss :  142.894800169
Epoch :  10 Step :  36 loss :  104.399908921
Epoch :  10 Step :  37 loss :  118.348122038
Epoch :  10 Step :  38 loss :  123.774543659
Epoch :  10 Step :  39 loss :  134.594286842
Epoch :  10 Step :  40 loss :  112.079634211
Epoch :  10 Step :  41 loss :  120.168524689
Epoch :  10

Epoch :  10 Step :  201 loss :  102.096150298
Epoch :  10 Step :  202 loss :  108.30094225
Epoch :  10 Step :  203 loss :  105.336903249
Epoch :  10 Step :  204 loss :  119.988053094
Epoch :  10 Step :  205 loss :  130.199818385
Epoch :  10 Step :  206 loss :  117.862630048
Epoch :  10 Step :  207 loss :  106.07302636
Epoch :  10 Step :  208 loss :  88.6394398211
Epoch :  10 Step :  209 loss :  110.804021322
Epoch :  10 Step :  210 loss :  115.798836691
Epoch :  10 Step :  211 loss :  123.04286415
Epoch :  10 Step :  212 loss :  98.4832722737
Epoch :  10 Step :  213 loss :  133.37063371
Epoch :  10 Step :  214 loss :  140.158349498
Epoch :  10 Step :  215 loss :  114.237765516
Epoch :  10 Step :  216 loss :  130.882794595
Epoch :  10 Step :  217 loss :  106.955192888
Epoch :  10 Step :  218 loss :  130.997083414
Epoch :  10 Step :  219 loss :  147.583309329
Epoch :  11 Step :  0 loss :  103.717368591
Epoch :  11 Step :  1 loss :  116.853704565
Epoch :  11 Step :  2 loss :  120.55315769

Epoch :  11 Step :  162 loss :  112.247104758
Epoch :  11 Step :  163 loss :  109.841597285
Epoch :  11 Step :  164 loss :  89.7987439662
Epoch :  11 Step :  165 loss :  119.953684642
Epoch :  11 Step :  166 loss :  96.7929440101
Epoch :  11 Step :  167 loss :  114.977573312
Epoch :  11 Step :  168 loss :  113.870559985
Epoch :  11 Step :  169 loss :  102.617640426
Epoch :  11 Step :  170 loss :  113.84282777
Epoch :  11 Step :  171 loss :  105.599494479
Epoch :  11 Step :  172 loss :  98.267438726
Epoch :  11 Step :  173 loss :  103.087942128
Epoch :  11 Step :  174 loss :  101.379234677
Epoch :  11 Step :  175 loss :  127.248947513
Epoch :  11 Step :  176 loss :  117.364139903
Epoch :  11 Step :  177 loss :  112.879452203
Epoch :  11 Step :  178 loss :  106.125282806
Epoch :  11 Step :  179 loss :  158.8325551
Epoch :  11 Step :  180 loss :  120.358749079
Epoch :  11 Step :  181 loss :  86.7171551057
Epoch :  11 Step :  182 loss :  123.559407525
Epoch :  11 Step :  183 loss :  114.48

Epoch :  12 Step :  123 loss :  113.951072432
Epoch :  12 Step :  124 loss :  98.5700788842
Epoch :  12 Step :  125 loss :  108.272946441
Epoch :  12 Step :  126 loss :  109.7693557
Epoch :  12 Step :  127 loss :  85.1038342278
Epoch :  12 Step :  128 loss :  103.918113308
Epoch :  12 Step :  129 loss :  128.850256532
Epoch :  12 Step :  130 loss :  73.8910296433
Epoch :  12 Step :  131 loss :  143.686233124
Epoch :  12 Step :  132 loss :  90.9639548354
Epoch :  12 Step :  133 loss :  132.833486391
Epoch :  12 Step :  134 loss :  135.11754362
Epoch :  12 Step :  135 loss :  143.18866327
Epoch :  12 Step :  136 loss :  118.041355287
Epoch :  12 Step :  137 loss :  107.851498783
Epoch :  12 Step :  138 loss :  106.972503148
Epoch :  12 Step :  139 loss :  122.151196703
Epoch :  12 Step :  140 loss :  88.2747233649
Epoch :  12 Step :  141 loss :  122.263121971
Epoch :  12 Step :  142 loss :  117.397280359
Epoch :  12 Step :  143 loss :  103.26283295
Epoch :  12 Step :  144 loss :  146.782

In [ ]:
(len(validating_data)-np.count_nonzero(model.predict(validating_data)-validating_data_class))/5000.0

In [ ]:
dumpModel(model)

In [12]:
testing_data=test.iloc[:,1:785].copy().as_matrix()/255.0
#testing_data=clf.transform(testing_data)

In [13]:
test_results=model.predict(testing_data)

In [14]:
tmp=[i for i in range(10000)]

In [15]:
csv_result=np.array(zip(tmp,test_results))

In [16]:
print csv_result

[[   0    4]
 [   1    4]
 [   2    2]
 ..., 
 [9997    3]
 [9998    7]
 [9999    3]]


In [17]:
np.savetxt('./res.csv',csv_result,delimiter=',',fmt='%d')